## 실습 [9-1]  

### 1. 실습명 : N-gram 언어 모델로 문장 생성하기
### 2. 실습 목적 및 설명
 
*   파이썬의 NLTK 패키지를 이용하여 N-gram 언어 모델을 구축한다
*   네이버에서 오픈 소스로 제공하는 nsmc 영화 리뷰 데이터셋을 이용해 문장을 생성한다.

### 3. 관련 장(챕터) : 9.2.2 N-gram 언어 모델 (N-gram Language Model)
### 4. 코드

In [20]:
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
import numpy as np
import codecs
from tqdm import tqdm
import random

In [21]:
# 한국어 처리에 필요한 konlpy 패키지를 설치하기 전에 선행 파일을 설치한다. 
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev

!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

from konlpy.tag import Okt

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [22]:
# NLTK 패키지를 이용하여 입력 텍스트를 N-gram 형태로 변환한다.
sentence = "나는 일본에 취업 하여 블록 체인 기술을 배워보고 싶다"

In [23]:
# NLTK 사용을 위하여 선행 패키지를 설치한다.
nltk.download('punkt')

# 입력 텍스트를 띄어쓰기 기준으로 토큰화한다.
tokens = word_tokenize(sentence) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
print(tokens)

['나는', '일본에', '취업', '하여', '블록', '체인', '기술을', '배워보고', '싶다']


In [25]:
# 한국어의 단어는 띄어쓰기를 기준으로 하지 않기 때문에 konlpy를 이용해 형태소를 기준으로 토큰화한다.
tagger = Okt()

def tokenize(text):
  tokens = ['/'.join(t) for t in tagger.pos(text)]
  return tokens

tokens = tokenize(sentence)
print(tokens)

['나/Noun', '는/Josa', '일본/Noun', '에/Josa', '취업/Noun', '하여/Verb', '블록/Noun', '체인/Noun', '기술/Noun', '을/Josa', '배워/Verb', '보고/Noun', '싶다/Verb']


In [26]:
# 토큰을 N-gram의 형태로 바꾸어준다. 
# ngrams 함수의 두 번째 인자로 N값을 지정할 수 있다.
# ngrams 함수는 문자열을 공백으로 구분하여 단어 단위 N개로 출력합니다
bigram = ngrams(tokens, 4)
trigram = ngrams(tokens, 3)

In [27]:
# N-gram을 출력해본다.
print("bigram: ")
for b in bigram:
  print(b)

print("\ntrigram: ")
for t in trigram:
  print(t) 

bigram: 
('나/Noun', '는/Josa', '일본/Noun', '에/Josa')
('는/Josa', '일본/Noun', '에/Josa', '취업/Noun')
('일본/Noun', '에/Josa', '취업/Noun', '하여/Verb')
('에/Josa', '취업/Noun', '하여/Verb', '블록/Noun')
('취업/Noun', '하여/Verb', '블록/Noun', '체인/Noun')
('하여/Verb', '블록/Noun', '체인/Noun', '기술/Noun')
('블록/Noun', '체인/Noun', '기술/Noun', '을/Josa')
('체인/Noun', '기술/Noun', '을/Josa', '배워/Verb')
('기술/Noun', '을/Josa', '배워/Verb', '보고/Noun')
('을/Josa', '배워/Verb', '보고/Noun', '싶다/Verb')

trigram: 
('나/Noun', '는/Josa', '일본/Noun')
('는/Josa', '일본/Noun', '에/Josa')
('일본/Noun', '에/Josa', '취업/Noun')
('에/Josa', '취업/Noun', '하여/Verb')
('취업/Noun', '하여/Verb', '블록/Noun')
('하여/Verb', '블록/Noun', '체인/Noun')
('블록/Noun', '체인/Noun', '기술/Noun')
('체인/Noun', '기술/Noun', '을/Josa')
('기술/Noun', '을/Josa', '배워/Verb')
('을/Josa', '배워/Verb', '보고/Noun')
('배워/Verb', '보고/Noun', '싶다/Verb')


In [28]:
# padding을 통해 입력 데이터에 문장의 시작과 끝을 알리는 토큰을 추가한다. 
bigram = ngrams(tokens, 4, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
print("bigrams with padding: ")
for b in bigram:
  print(b)

bigrams with padding: 
('<s>', '<s>', '<s>', '나/Noun')
('<s>', '<s>', '나/Noun', '는/Josa')
('<s>', '나/Noun', '는/Josa', '일본/Noun')
('나/Noun', '는/Josa', '일본/Noun', '에/Josa')
('는/Josa', '일본/Noun', '에/Josa', '취업/Noun')
('일본/Noun', '에/Josa', '취업/Noun', '하여/Verb')
('에/Josa', '취업/Noun', '하여/Verb', '블록/Noun')
('취업/Noun', '하여/Verb', '블록/Noun', '체인/Noun')
('하여/Verb', '블록/Noun', '체인/Noun', '기술/Noun')
('블록/Noun', '체인/Noun', '기술/Noun', '을/Josa')
('체인/Noun', '기술/Noun', '을/Josa', '배워/Verb')
('기술/Noun', '을/Josa', '배워/Verb', '보고/Noun')
('을/Josa', '배워/Verb', '보고/Noun', '싶다/Verb')
('배워/Verb', '보고/Noun', '싶다/Verb', '</s>')
('보고/Noun', '싶다/Verb', '</s>', '</s>')
('싶다/Verb', '</s>', '</s>', '</s>')


In [29]:
# 문장 생성을 위하여 네이버 영화 리뷰 데이터셋을 다운로드한다.
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

CPU times: user 10.6 ms, sys: 15 ms, total: 25.6 ms
Wall time: 143 ms


In [30]:
# 다운로드 받은 데이터셋을 읽고 인덱스와 라벨을 제외한 텍스트 부분만 가져온다.
# codecs 패키지는 대용량 파일을 조금씩 읽을 수 있게 해준다. 

with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()] # \n 제외
  data = data[1:] # header 제외
print("데이터셋: ", data[:5])
docs = [row[1] for row in data] # 텍스트 부분만 가져옴
print("\n텍스트 데이터:", docs[:3])
print("\n문장 개수: ",len(docs)) # 총 15만개의 문장으로 이루어진 데이터셋임을 알 수 있다.

데이터셋:  [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1']]

텍스트 데이터: ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다']

문장 개수:  150000


In [ ]:
# 토큰화한 텍스트 데이터의 bigram을 모두 리스트에 추가한다.
sentences = []
for d in tqdm(docs):
  tokens = tokenize(d)
  bigram = ngrams(tokens, 4, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
  sentences += [t for t in bigram]


100%|██████████| 150000/150000 [10:21<00:00, 241.22it/s]


In [ ]:
print(sentences[10:15])

[('<s>', '<s>', '흠/Noun', '.../Punctuation'), ('<s>', '흠/Noun', '.../Punctuation', '포스터/Noun'), ('흠/Noun', '.../Punctuation', '포스터/Noun', '보고/Noun'), ('.../Punctuation', '포스터/Noun', '보고/Noun', '초딩/Noun'), ('포스터/Noun', '보고/Noun', '초딩/Noun', '영화/Noun')]


In [ ]:
cfd = ConditionalFreqDist(sentences)
print(cfd["<s>"].most_common(4))

ValueError: ignored

In [ ]:
# 주어진 토큰(c) 다음에 가장 많이 등장하는 n개의 단어를 반환하는 함수를 만든다.
def most_common(c, n, pos=None):
  if pos is None:
    return cfd[tokenize(c)[0]].most_common(n)
  else:
    return cfd["/".join([c, pos])].most_common(n)

In [ ]:
print(most_common("나", 10))

In [ ]:
# 단어별 등장 빈도를 기반으로 조건부 확률을 추정한다.
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [ ]:
# “.” 다음에 “</s>”가 올 확률을 출력한다.
print(cpd[tokenize(".")[0]].prob("</s>"))

In [ ]:
# 토큰 c 다음에 토큰 w가 bigram으로 함께 등장할 확률을 구한다.
def bigram_prob(c, w):
  context = tokenize(c)[0]
  word = tokenize(w)[0]
  return cpd[context].prob(word)

In [ ]:
print(bigram_prob("이", "영화"))

In [ ]:
print(bigram_prob("영화", "이"))

In [ ]:
# 조건부 확률을 알게 되면 가장 확률이 높은 토큰열을 토대로 문장을 생성할 수 있다.
def generate_sentence(seed=None, debug=False):
  if seed is not None:
    import random
    random.seed(seed)
  c = "<s>"
  sentence = []
  while True:
    if c not in cpd:
      break
    w = cpd[c].generate()

    if w == "</s>":
      break
    
    word = w.split("/")[0]
    pos = w.split("/")[1]

    # 조사, 어미 등을 제외하고 각 토큰은 띄어쓰기로 구분하여 생성한다.
    if c == "<s>":
      sentence.append(word.title())
    elif c in ["`", "\"","'","("]:
      sentence.append(word)
    elif word in ["'", ".", ",", ")", ":", ";", "?"]:
      sentence.append(word)
    elif pos in ["Josa", "Punctuation", "Suffix"]:
        sentence.append(word)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
        sentence.append(word)
    else:
        sentence.append(" " + word)
    c = w

    if debug:
      print(w)

  return "".join(sentence)

In [ ]:
print(generate_sentence(2))

In [ ]:
generate_sentence(2, debug=True)

<참고>

> https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/


